# Step 2: Slice-Timing

As we know, when it comes to acquiring a whole volume of the brain the scanner cannot do this instantaneously. Instead, the data are acquired for one slice of the image at a time, as shown in {numref}`slice-order-fig`. These slice acquisition sequences can be any order, but typically are either sequential (e.g. from the bottom to the top of the head) or interleaved (e.g. all odd slices first then all even slices). So it takes a single TR to collect a whole volume, meaning the slices within that volume are all collected at different time-points. For fMRI, this means that the time when the BOLD signal was measured is going to be different in each slice. Because a functional region of the brain may span multiple slices, this means that the point during that region's response that is measured is going to depend on which slice the region falls in. For an interleaved sequence, this could mean a difference of 1/2 TR within the same region. This timing discrepancy can cause serious problems when it comes to the statistical modelling of the data and thus we usually need to try and do something to correct this before reaching the modelling stage.

```{figure} images/slice-order.jpg
---
width: 400px
name: slice-order-fig
---
Illustration of how a single volume of the brain consists of multiple slices collected at different times.
```

## The slice timing problem

The main issue with slice-timing is that, in terms of measuring the BOLD signal, even if the shape of the signal is the same in different slices, different points will be measured due to the time delay from one slice to the next. This means that the measured shape can actually be quite different from the true shape of the signal. This is illustrated in {numref}`slice-timing-fig` from Sladky et al. (2011). The top 5 rows represent the true BOLD signal in different slices with the red dots representing the points measured by the scanner. The first slice is measured at the start of the TR and then each subsequent slice is measured slightly later. Using those sampled points to reconstruct the signal leads to quite different shapes in slices that were measured later, as shown in the bottom 5 rows. This differences is extreme enough that the signal in slice 5 actually looks like it peaked a TR earlier than it actually did.

```{figure} images/slice-timing.png
---
width: 600px
name: slice-timing-fig
---
Illustration of the slice-timing problem.
```

This is a problem because when we come to create our statistical model of the data, we normally assume that the shape of the signal will look like the first slice. So in this example we would assume that, based on our stimulus presentation, the peak of the signal will occur around TR 6 and TR 9. This will work well for the first slice, but will not work for the last slice. This will make more sense when we get to the statistical modelling, but for now just known that the model we use implicitly assumes that all the data were acquired simultaneously, which clearly is not true. So what is the solution? One approach would be to change the model for each slice to reflect the timing offset. This is an approach that's been used successfully in the past (see Worsley et al., 2002), but is not the method SPM uses. Instead, SPM takes the approach of changing the data in each slice using a method known as slice-timing correction.

## Slice-timing correction

As its most basic, slice-timing correction is a method of interpolating the timeseries data at a voxel forwards or backwards in time so that it represents the signal if it had been measured earlier or later by the scanner. Doing this for each slice means we can create a dataset where all the timeseries data appears as if it were collected simultaneously. In order to do this, you need knowledge of what order the slices were collected in, and then which slice you want to use as the reference. This reference slice represents the time you want to interpolate all other slices to match. Intuitively, you may think that interpolating all the data to match the first slice makes sense, but this does mean that more interpolation is need for later slices (nearly a whole TR) and so often the middle slice is chosen to reduce this. This means the whole dataset is interpolated to appear as if it were collected halfway between each TR.

This process is illustrated in the figure below from Poldrack, Mumford and Nichols (2011). The blue line represents the original measured signal. The peaks in the blue line represent the originally measured values and the red circles represent the points interpolated back in time by half a TR. The dotted red line therefore represents the new signal after slice-timing correction. Of note is that this example uses linear interpolation, and so the points back in time are based on drawing a straight line between the original sampled points. In reality, slice-timing uses more sophisticated interpolation methods to provide a more accurate guess at what the signal looked like earlier/later in time.

```{figure} images/slice-timing-correction.png
---
width: 800px
name: slice-timing-corr-fig
---
Illustration of slice-timing correction.
```

From a practical perspective, in order to perform slice-timing you need to know the type of acquisition sequence used to collect the data. If it is your own dataset, then you should know this from when the scans were setup. However, if it is a dataset you have acquired from elsewhere, this information may not be available. Please remember that if you do not have this information you cannot perform slice-timing correction. Any attempt to simply guess what the sequence was will result in garbage, as the interpolation will not make any sense.

## Slice-timing controversy

Unlike all the other pre-processing steps in this lesson, slice-timing correction is quite a contentious step. One of the more general issues is because slice-timing correction actually changes the shape of the timeseries at each voxel. As we will come to see, this shape is a large part of what we model and thus any changes to this shape are almost certainly going to affect our results. This means we have to be pretty confident that we are changing things accurately and not introducing any changes that create spurious results. This alone may make you feel uncomfortable about performing slice-timing correction on your data.

More specifically, the issues with slice-timing come down to its interactions with motion, and the fact that the interpolation methods employed by SPM can caused a spreading of artifacts in the data across multiple slices. Of these two, the interaction with motion is a particularly tricky element of slice-timing as depending on the amount of motion and the acquisition sequence the ordering of motion correction and slice-timing may be better reversed. In this lesson we have suggested motion correction first and slice-timing second, however, this is not a universal recommendation. In addition, the degree to which slice-timing is necessary depends upon the length of the TR. Clearly a discrepancy between slices of up to 5 seconds is a bigger issue than a discrepancy of 1 or 2 seconds. Indeed, there are methods we can employ during the statistical modelling that allows us to introduce some elements of automatic temporal adjustment that could be used instead of slice-timing correction. We will pick these points up again at the end of the lesson where we will explore building the pre-processing pipeline in more detail.

## Slice-timing in SPM

The video below will show you how to perform slice-timing correction on the realigned functional scans, using SPM12.

<div style="max-width: 1280px"><div style="position: relative; padding-bottom: 56.25%; height: 0; overflow: hidden;"><iframe src="https://livemanchesterac.sharepoint.com/sites/UOM-FBMH-MSc-NCCN/_layouts/15/embed.aspx?UniqueId=c1273482-26be-488d-9b18-d308de89f0df&embed=%7B%22hvm%22%3Atrue%2C%22ust%22%3Atrue%7D&referrer=StreamWebApp&referrerScenario=EmbedDialog.Create" width="1280" height="720" frameborder="0" scrolling="no" allowfullscreen title="4.Slice-Timing.mov" style="border:none; position: absolute; top: 0; left: 0; right: 0; bottom: 0; height: 100%; max-width: 100%;"></iframe></div></div>